<a href="https://colab.research.google.com/github/Spanholz/GaussianBeamProfile/blob/main/Gaussian_Energy_Distribution_in_focal_spot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
#@title

# Imports
import os
import sys
import math

import numpy as np              # Numpy import
import matplotlib.pyplot as plt # Pyplot import
import matplotlib.cm as cm
import matplotlib.ticker as ticker

from IPython.display import clear_output

import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, Output
from ipywidgets import interact, interact_manual


#from Intensity import intensity # Import of Intensity calculation

from matplotlib.patches import Ellipse # Ellipse for displaying the measured voxel size

## Values in X, Y, Z the intensity will be calculated for

def createMesh(dxy, dz):
    steps = di.value # Number of points in the calculated meshgrid

    x_vals = np.linspace(-dxy/2, dxy/2, steps)
    y_vals = np.linspace(-dxy/2, dxy/2, steps)
    z_vals = np.linspace(-dz/2, dz/2, steps) 

    x, z = np.meshgrid(x_vals, z_vals)
    x, y = np.meshgrid(x_vals, y_vals)

    y = np.array(0) # Set y to zero for XZ plot

    XYZ = x, y, z


    return XYZ

    #@title

# Calculation of intensities
def intensity(constants, XYZ):

    wl, tau, f, NA, M, NA, P = constants
    x, y, z = XYZ

    w0 = (M**2 * 0.61 * wl) / NA # Beam waist size at focus
    zR = (math.pi * w0**2)/wl # Rayleigh range

    wzc = w0 * (1 + (z/zR)**2)**0.5  # Evolving beam width from z=0 focus point
    

    I0 = (2 * P * M)/(f * w0**2 * math.pi * tau) # Intensity at center peak

    # Intensity at a given point - depending on X, Y, Z

    I   = I0 * (w0/wzc)**2 * np.exp(-2 * (((x**2 + y**2)**0.5)**2)/(wzc**2))
    return I

def plotEnergy(I, XYZ):
    #Plotting
    x, y, z = XYZ
    # setup figures
    fig, ax1 = plt.subplots(figsize=(10,8))


    # Micrometer instead of meter in scales
    scale_x = 1e-6        # Microns 
    scale_y = 1e-6        # Microns
    scale_colorbar = 1e14 # Terrawatts
    ticks_x = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_x)) # Rescale the ticks to µm
    ticks_colorbar = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_colorbar)) # Rescale the colorbar to Terrawatts


    ax1.xaxis.set_major_formatter(ticks_x)
    ax1.yaxis.set_major_formatter(ticks_x)

    ax1.set_aspect('equal', 'box')
    ax1.set_xlabel("X" + " in µm")
    ax1.set_ylabel("Z" + " in µm")
    ax1.set_title('Intensity in focal spot with ' + str(NA.value) + " NA, Power: " + str(P.value) + " mW")


    # Draws a contour line around the actual polymerized area with a certain intensity

    #treshold = 14e14
    if contourLines.value == True:
        con = ax1.contour(x, z, I)
                      #, levels = [treshold]
                      #, colors=('k',), linestyles=('-',),linewidths=(2,))
    # # Draws the actual intensity plot overthe figuregrid area

    conf = ax1.contourf(x, z,  I, levels = 200, cmap = "jet") # Making a colourmap
    cb = fig.colorbar(conf, ax=ax1) # Make a colorbar next to the intensity plot
    cb.ax.yaxis.set_major_formatter(ticks_colorbar) # Recalculate the ticks to TW/cm2
    cb.ax.set_ylabel("TW/cm²") # Set label of colorbar

    #filename = 'Intensity_' + str(NA.value) + "NA_Power" + str(P.value) + "mW" + ".png"
    plt.show()
    #print(filename)
    #plt.savefig(filename, dpi=150)

    #@title


NA = widgets.Dropdown(
    value=1.4, 
    options=[1.4, 0.95, 0.8, 0.7, 0.45], 
    #description='Numerical Aperture'
)

P = widgets.IntSlider(
    min=0,
    max=60,
    value=10,
    step=1
    #description="Power in mW"
)

M = widgets.FloatSlider(
    min=1,
    max=1.8,
    value=1.2,
    step=0.1
    #description="M² value"
)

wl = widgets.IntText(
    value=780,
    #description='Wavelength in nm:',
)

tau = widgets.IntText(
    value=100
    #description='Pulse duration in fs:',
)

f = widgets.IntText(
    value=100
    #description='Laser repetition rate in MHz:',
)

dtxy = widgets.FloatSlider(
    min = 0.5,
    max = 50,
    value=6,
    step = 0.5
    #description='Range in XY in µm:',
)

dtz = widgets.FloatSlider(
    min = 0.5,
    max = 50,
    value=6,
    step = 0.5
    #description='Range in Z in µm:',
)

di = widgets.IntSlider(
    min = 100,
    max = 2000,
    value=500,
    step = 20,
    #description='Amount of points in XY to calculate',
)


contourLines = widgets.Checkbox()

calc_button = Button(description='Calculate',
                     button_style="primary")

def on_button_clicked(b):
    NA_num = NA.value
    P_num = P.value  * 10**-3
    M_num = M.value
    wl_num = wl.value * 10**-9
    tau_num = tau.value * 10**-15
    f_num = f.value * 10**6
    constants = wl_num, tau_num, f_num, NA_num, M_num, NA_num, P_num
    dtxy_num = dtxy.value * 10**-6
    dtz_num = dtz.value * 10**-6
    
    XYZ = createMesh(dtxy_num, dtz_num)
    I = intensity(constants, XYZ)

    with out:
        clear_output(wait=True)
        plotEnergy(I, XYZ)
        print("Create Mesh of Data points")
        print("Intensity is calculated. This may take a few seconds.")

    return constants
    

    

calc_button.on_click(on_button_clicked)

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_button_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
)

form_items = [
     Box([Label(value='Numerical aperture'), NA], layout=form_item_layout),
     Box([Label(value='Laser Power in mW'), P], layout=form_item_layout),
     Box([Label(value='M-value - closeness to a gaussian beam'), M], layout=form_item_layout),
     Box([Label(value='Laser Wavelength in nm'), wl], layout=form_item_layout),
     Box([Label(value='Pulse duration in fs'), tau], layout=form_item_layout),
     Box([Label(value='Laser repetition rate in MHz'), f], layout=form_item_layout),
     Box([Label(value='Calculated Size in XY'), dtxy], layout=form_item_layout),
     Box([Label(value='Calculated Size in Z'), dtz], layout=form_item_layout),
     Box([Label(value="Contour Lines"), contourLines],layout=form_item_layout), 
     Box([Label(value= "Amount of points in XY to calculate"), di], layout=form_button_layout),
     Box([Label(value="Start Calculation:"), calc_button], layout=form_button_layout)
     ]


form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))

out = widgets.Output()

display(form)
display(out)

Box(children=(Box(children=(Label(value='Numerical aperture'), Dropdown(options=(1.4, 0.95, 0.8, 0.7, 0.45), v…

Output()